# * Hyperspectral Imageları kalibre ederek bant boyutunu azaltma ve hasta klasörlerine kayıt etme *

In [8]:
import os
import shutil
import tensorflow as tf
import numpy as np
import spectral.io.envi as envi
from scipy.ndimage import uniform_filter, gaussian_filter
from sklearn.decomposition import PCA #,FastICA


# GPU'nun kullanılabilirliğini kontrol et ve aktif yap

In [9]:
# tensorflow versiyon kontrolü
print("TensorFlow versiyonu:", tf.__version__)


gpu_devices = tf.config.list_physical_devices('GPU')
print("Kullanılabilir GPU sayısı: ", len(tf.config.list_physical_devices('GPU')))
if gpu_devices:
    print("Kullanılabilir GPU:", gpu_devices)
    tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')
    try:
        # Dinamik bellek tahsisi ayarla
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
        print("Dinamik Bellek Kullanımı Ayarlandı!")
    except RuntimeError as e:
        print(e)
else:
    print("GPU bulunamadı")


TensorFlow versiyonu: 2.10.1
Kullanılabilir GPU sayısı:  1
Kullanılabilir GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Dinamik Bellek Kullanımı Ayarlandı!


# Kalibrasyon ve filtreleme fonksiyonları

In [10]:
# Kalibrasyon fonksiyonu
def calibrate_hyperspectral_data(raw, dark_ref, white_ref):
    return (raw - dark_ref) / (white_ref - dark_ref)

def noise(data, filter_size=5):
    # Uzaysal filtreleme: her bant için uygulanır    
    return uniform_filter(data, size=filter_size)
    """
    filtered_data = np.empty_like(data)
    for i in range(data.shape[2]):  # Bantlar boyunca dön
        filtered_data[:, :, i] = uniform_filter(data[:, :, i], size=filter_size)
    return filtered_data
    """

# Spektral Düzeltme (Gaussian Smoothing)
def gauss(data, sigma=1):    
    return gaussian_filter(data, sigma=(0, 0, sigma))

# Normalize Etme
def normalize(data):    
    """filtered_data = np.empty_like(data)
    for i in range(data.shape[2]):  # Bantlar boyunca dön
        min_val = np.min(data[:, :, i])
        max_val = np.max(data[:, :, i])
        filtered_data[:, :, i] = (data[:, :, i] - min_val) / (data[:, :, i] - min_val+ 1e-9)    
    return filtered_data
    """
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val+ 1e-9)    


# Image gruplama foksiyonları

In [11]:
def reduce_spectral_bands(image, target_depth):
    """
    Hiperspektral görüntüyü PCA kullanarak hedef bant sayısına indirger.

    Args:
        image (np.ndarray): Hiperspektral görüntü (row, col, bands).
        target_depth (int): Hedef spektral bant sayısı.

    Returns:
        np.ndarray: PCA ile indirgenmiş spektral bantlar.
    """
    # Orijinal boyutlar
    rows, cols, bands = image.shape
    image_reshaped = image.reshape(-1, bands)  # (row*col, bands)

    # PCA ile indirgeme
    pca = PCA(n_components=target_depth)
    reduced_data = pca.fit_transform(image_reshaped)  # (row*col, target_depth)

    # Yeniden şekillendirme
    reduced_image = reduced_data.reshape(rows, cols, target_depth)
    return reduced_image
    
def pcagrupla(hyperspectral_image, wavelength, bandgrup=5):
    """
    HSI verisini PCA ile bantları birleştirerek azaltır ve yeni bir wavelength listesi döner.

    Args:
    - hyperspectral_image: Hyperspectral görüntü (yükseklik, genişlik, bant sayısı)
    - wavelength: Orijinal wavelength değerlerinin listesi
    - bandgrup: Birleştirilecek bant sayısı

    Returns:
    - new_image: PCA ile indirgenmiş yeni görüntü
    - new_wavelength: Yeni wavelength listesi (her grubun ilk ve son dalga boyları birleştirilmiş)
    """
    height, width, num_bands = hyperspectral_image.shape

    # Her bandgrup bandı birleştirip PCA ile 1'e düşür
    bands_to_combine = bandgrup
    new_num_bands = num_bands // bands_to_combine  # Yeni bant sayısı
    new_image = np.zeros((height, width, new_num_bands))
    new_wavelength = []

    for i in range(new_num_bands):
        # Bant aralığını al
        start_band = i * bands_to_combine
        end_band = start_band + bands_to_combine
        image = hyperspectral_image[:, :, start_band:end_band]

        # Wave length aralığını birleştir (ilk ve son değer)
        group_wavelength = f"{wavelength[start_band]:.2f}-{wavelength[end_band-1]:.2f}"
        new_wavelength.append(group_wavelength)

        # PCA uygulama
        image_reshaped = image.reshape(-1, bandgrup)  # (row*col, bands)
        pca = PCA(n_components=1)
        reduced_data = pca.fit_transform(image_reshaped)  # (row*col, target_depth)
        
        # İndirgenmiş veriyi yeni banda ekle
        new_image[:, :, i] = reduced_data.reshape(height, width)

    return new_image, new_wavelength    


    
def parcagrupla(hyperspectral_image,wavelength, bandgrup=5):
    """
    HSI verisini belirtilen bant grupta bir bant alır. Yeni bir image ve wavelength listesi döner.

    Args:
    - hyperspectral_image: Hyperspectral görüntü (yükseklik, genişlik, bant sayısı)
    - wavelength: Orijinal wavelength değerlerinin listesi
    - bandgrup: Birleştirilecek bant sayısı

    Returns:
    - new_image: Her bantgrupla  alınan ve indirgenmiş yeni görüntü
    - new_wavelength: Yeni wavelength listesi (her grubun ilk ve son dalga boyları birleştirilmiş)
    """  
    
  height, width, num_bands = hyperspectral_image.shape

  # Her bandgrup bandı birleştiripaliıp birlestirerek yeni bir görüntü oluştur
  bands_to_combine = bandgrup
  new_num_bands = num_bands // bands_to_combine  # Yeni bant sayısı
  new_image = np.zeros((height, width, new_num_bands))
  new_wavelength = []

  for i in range(new_num_bands):
      # Bant aralığını al
      start_band = i * bands_to_combine      

      new_wavelength.append(f"{wavelength[start_band]:.2f}")

      # yeni banda ekle
      new_image[:, :, i] = hyperspectral_image[:, :, start_band]

  
  return new_image,new_wavelength

def ortalamagrupla(hyperspectral_image,wavelength,bandgrup=5):
    """
    HSI verisini her bantgrup imageyi Ortalama ile bantları birleştirerek azaltır ve yeni bir wavelength listesi döner.

    Args:
    - hyperspectral_image: Hyperspectral görüntü (yükseklik, genişlik, bant sayısı)
    - wavelength: Orijinal wavelength değerlerinin listesi
    - bandgrup: Birleştirilecek bant sayısı

    Returns:
    - new_image: ORtalama ile indirgenmiş yeni görüntü
    - new_wavelength: Yeni wavelength listesi (her grubun ilk ve son dalga boyları birleştirilmiş)
    """
  # Örnek hyperspectral görüntü (boyut: [height, width, num_bands])
  # Kendi verinizi buraya yükleyin
  height, width, num_bands = hyperspectral_image.shape

  # Her bandgrup bandı birleştirip ortalamasını alarak yeni bir görüntü oluştur
  bands_to_combine = bandgrup
  new_num_bands = num_bands // bands_to_combine  # Yeni bant sayısı
  new_image = np.zeros((height, width, new_num_bands))
  new_wavelength = []
    
  for i in range(new_num_bands):
      # Bant aralığını al
      start_band = i * bands_to_combine
      end_band = start_band + bands_to_combine

      # Wave length aralığını birleştir (ilk ve son değer)
      group_wavelength = f"{wavelength[start_band]:.2f}-{wavelength[end_band-1]:.2f}"
      new_wavelength.append(group_wavelength)      

      # Ortalama alarak yeni banda ekle
      new_image[:, :, i] = np.mean(hyperspectral_image[:, :, start_band:end_band], axis=2)

  
  return new_image,new_wavelength

# Image işleme fonksiyonları

In [15]:
def imgsave(destination_folder,filename,image,waves,metadata={}):
    # Image yi belirtilen hedefe kayıt eder
    
    # Çıktı dosyası adı
    hdr = os.path.join(destination_folder, filename)
    
    metadata['wavegroup'] = waves
    
    # Kalibre edilmiş veriyi kaydet
    envi.save_image(
        hdr,
        image,
        metadata=metadata,
        dtype=np.float32,
        interleave="bil",  # Band Interleave
        force=True  # Üzerine yazmayı etkinleştir
    )          

# Ana işlem fonksiyonu
def process_dataset(dataset_path,destination_path,parca=5):     
    
    # Belirtilen yolda Tüm alt klasörleri gez
    for root, dirs, files in os.walk(dataset_path):
        # Kalibrasyon için gerekli tüm dosyalar var ise işleme geç
        if "darkReference.hdr" in files and "whiteReference.hdr" in files and "raw.hdr" in files:
            print(f"Processing folder: {root}")

            # dataset_path'i çıkararak göreli yolu (Hasta klasörü adını) al
            relative_path = os.path.basename(root)

            # Hedef yolda göreli yolu birleştir
            destination_folder = os.path.join(destination_path, relative_path)

            # Hedef klasör yoksa oluştur
            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder)
                print(f"Hedef klasör oluşturuldu: {destination_folder}")

            # Dosya yollarını oluştur
            dark_hdr = os.path.join(root, "darkReference.hdr")
            white_hdr = os.path.join(root, "whiteReference.hdr")
            raw_hdr = os.path.join(root, "raw.hdr")

            # Verinin bulunduğu uzantısız dosyaları belirle
            dark_data_path = dark_hdr.replace(".hdr", "")  # Uzantısız veri dosyası
            white_data_path = white_hdr.replace(".hdr", "")  # Uzantısız veri dosyası
            raw_data_path = raw_hdr.replace(".hdr", "")  # Uzantısız veri dosyası

            # gtMap dosyalarını hedef klasöre kopyala
            # 0 dan olusturulur iken burasını açıp çalıştır
            
            # gtmap_hdr_source=os.path.join(root, "gtMap.hdr")
            # gtmap_hdr_destination = os.path.join(destination_folder, "gtMap.hdr")
            # shutil.copy(gtmap_hdr_source, gtmap_hdr_destination)
            # gtmap_hdr_destination=gtmap_hdr_destination.replace(".hdr", "")
            # print(f"gtMap.hdr kopyalandı : {gtmap_hdr_destination}")
            # shutil.copy(gtmap_data_path, gtmap_hdr_destination)
            # print(f"gtMap kopyalandı: {gtmap_hdr_destination}")
            
            # ENVI formatındaki verileri yükle
            dark_ref = envi.open(dark_hdr, dark_data_path).load()
            white_ref = envi.open(white_hdr, white_data_path).load()
            raw_data = envi.open(raw_hdr, raw_data_path).load()

            # Kalibrasyon işlemi
            calibrated_data = calibrate_hyperspectral_data(raw_data, dark_ref, white_ref)

            # Aşırı Bant Gürültüsü Giderme (Extreme Band Noise Removal) ilk 56 son 126 makaleden alınan bilgi
            calibrated_data = calibrated_data[:, :, 56:-126]

            # Kırpılmış wavelength bilgilerini al            
            metadata = raw_data.metadata            
            original_wavelengths = np.array(metadata['wavelength'], dtype=float)
            
            #RGB image olusturmak icin belirtilen bant noları
            default_bands=np.array(metadata['default bands'], dtype=int)
            
            #calibrated_data ya kayıt sırasında eşleşen dalgaboyu degerlerini almak icin aynı kırpma islemi uggula
            cropped_wavelengths = original_wavelengths[56:-126].tolist()    

            # Metadata'yı güncelle
            metadata={}           
            # calibrated_data daki kırpmalardan dolayı RGB oluşturacak bant degerleri de aynı oranda kırpılmalı
            metadata['default bands'] =  [band - 56 for band in default_bands]

            imgsave(destination_folder,filename=f"calibrated_raw.hdr",image=calibrated_data,waves=cropped_wavelengths,metadata=metadata)

            # Filteleme işlemleri eğitim sırasında yapılacak
            
            # Gürültü Filtreleme
            # calibrated_data = noise_filtering(calibrated_data, filter_size=5)

            # Normalizasyon
            # calibrated_data = normalize_data(calibrated_data)

            # PCA belirtilen her parcalık bandı PCA islemi yaparak tek banda düşür ve ardından tüm PCA lı bantları birleştir ve yeni Image dosyasını kaydet
            
            pcaimage,pcawave=pcagrupla(calibrated_data,cropped_wavelengths,bandgrup=parca)
            pcawave=cropped_wavelengths            
            
            imgsave(destination_folder,filename=f"calibrated_pca_{parca}.hdr",image=pcaimage,waves=pcawave)

            # ORTALAMA belirtilen her parcalık bandı Ortalamasını yaparak tek banda düşür ve ardından tüm Ortalaması alınmış bantları birleştir ve yeni Image dosyasını kaydet
            
            ortimage,ortwave=ortalamagrupla(calibrated_data,cropped_wavelengths,bandgrup=parca)
            imgsave(destination_folder,filename=f"calibrated_ort_{parca}.hdr",image=ortimage,waves=ortwave)

            # PARCA belirtilen her parcadaki 1 bandı al sonrasında alınmış bantları birleştir ve yeni Image dosyasını kaydet
            
            parcaimage,parcawave=parcagrupla(calibrated_data,cropped_wavelengths,bandgrup=parca)            
            imgsave(destination_folder,filename=f"calibrated_par_{parca}.hdr",image=parcaimage,waves=parcawave)


            print(f"Calibrated data: {calibrated_data.shape} pca : {pcaimage.shape}  ort: {ortimage.shape} par: {parcaimage.shape} \n {destination_folder}")
            
            



# Image ları işle

In [16]:
# Dataset yolu
dataset_path = "D:/proje/Data/npj_database"
destination_path = "D:/proje/Data/hasta"

# Hedef yol yoksa oluştur
if not os.path.exists(destination_path):
    os.makedirs(destination_path)
    print(f"Hedef klasör yolu oluşturuldu : {destination_path}")
else:
    print(f"Hedef klasör yolu mevcut : {destination_path}")

# Dataseti işle
process_dataset(dataset_path,destination_path,parca=128)

Hedef klasör yolu mevcut : D:/proje/Data/hasta
Processing folder: D:/proje/Data/npj_database\FirstCampaign\004-02\004-02
Calibrated data: (389, 345, 644) pca : (389, 345, 128)  
 D:/proje/Data/hasta\004-02
Processing folder: D:/proje/Data/npj_database\FirstCampaign\005-01\005-01
Calibrated data: (483, 488, 644) pca : (483, 488, 128)  
 D:/proje/Data/hasta\005-01
Processing folder: D:/proje/Data/npj_database\FirstCampaign\007-01\007-01
Calibrated data: (582, 400, 644) pca : (582, 400, 128)  
 D:/proje/Data/hasta\007-01
Processing folder: D:/proje/Data/npj_database\FirstCampaign\008-01\008-01
Calibrated data: (460, 549, 644) pca : (460, 549, 128)  
 D:/proje/Data/hasta\008-01
Processing folder: D:/proje/Data/npj_database\FirstCampaign\008-02\008-02
Calibrated data: (480, 553, 644) pca : (480, 553, 128)  
 D:/proje/Data/hasta\008-02
Processing folder: D:/proje/Data/npj_database\FirstCampaign\010-03\010-03
Calibrated data: (371, 461, 644) pca : (371, 461, 128)  
 D:/proje/Data/hasta\010-03